In [1]:
import pandas as pd
df_resp = pd.read_csv('/project2/wparker/SIPA_data/RCLIF_respiratory_support_09282023.csv')
df_resp['recorded_time'] = pd.to_datetime(df_resp['recorded_time'], format='%Y-%m-%d %H:%M:%S')

df_resp = df_resp.loc[(df_resp['recorded_time']>='2020-03-01 00:00:00') & 
                      (df_resp['recorded_time']<='2022-03-31 11:59:59')]

In [2]:
#get just invasive or non-invasive mechanical ventilation
vent = df_resp.loc[(df_resp['device_name']=='Vent') | (df_resp['device_name']=='NIPPV')]

# minimum time by person

vent_grouped = vent.groupby('C19_HAR_ID')['recorded_time'].agg(['min'])
vent_grouped = vent_grouped.rename(columns={'min': 'min_resp_support_time'})

In [3]:
df_meds = pd.read_csv('/project2/wparker/SIPA_data/RCLIF_meds_admin_conti.csv')
df_resp['recorded_time'] = pd.to_datetime(df_resp['recorded_time'], format='%Y-%m-%d %H:%M:%S')

In [4]:
pressors = df_meds.loc[(df_meds['med_name']=='phenylephrine') | 
                       (df_meds['med_name']=='epinephrine') | 
                       (df_meds['med_name']=='vasopressin') | 
                       (df_meds['med_name']=='dopamine') |
                       (df_meds['med_name']=='dobutamine') |
                       (df_meds['med_name']=='norepinephrine') |
                       (df_meds['med_name']=='angiotensin') |
                       (df_meds['med_name']=='isoproterenol')]
pressors = pressors[["C19_HAR_ID", "admin_time"]]
print(len(pressors))
pressors_no_na = pressors.dropna()
print(len(pressors_no_na))
pressors_no_na['admin_time'] = pd.to_datetime(pressors_no_na['admin_time'], format='%Y-%m-%d %H:%M:%S')
pressors_no_na = pressors_no_na.loc[(pressors_no_na['admin_time']>='2020-03-01 00:00:00') & 
                      (pressors_no_na['admin_time']<='2022-03-31 11:59:59')]

pressors_grouped = pressors_no_na.groupby('C19_HAR_ID')['admin_time'].agg(['min'])
pressors_grouped = pressors_grouped.rename(columns={'min': 'min_pressor_time'})

216776
216129


/scratch/local/jobs/9871442/ipykernel_2654588/1479992001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pressors_no_na['admin_time'] = pd.to_datetime(pressors_no_na['admin_time'], format='%Y-%m-%d %H:%M:%S')


In [5]:
df = pressors_grouped.join(vent_grouped, on='C19_HAR_ID', how='outer')

In [6]:
df = df[['C19_HAR_ID','min_pressor_time','min_resp_support_time']]
df['min_pressor_time'] = pd.to_datetime(df['min_pressor_time'], format='%Y-%m-%d %H:%M:%S')
df['min_resp_support_time'] = pd.to_datetime(df['min_resp_support_time'], format='%Y-%m-%d %H:%M:%S')


df['life_support_start'] = df[['min_pressor_time','min_resp_support_time']].min(axis=1)

In [7]:
df = df[['C19_HAR_ID', 'life_support_start']]
df = df.loc[(df['life_support_start']>='2020-03-01 00:00:00') & 
                      (df['life_support_start']<='2022-03-31 11:59:59')]

In [8]:
from pathlib import Path 
filepath=Path('/project2/wparker/SIPA_data/life_support_cohort.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 

df.to_csv(filepath, index=False)